<a href="https://colab.research.google.com/github/zakaria-kabir/Research_Bengali-Dialect-Detection/blob/main/Audio_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Mounting**

In [16]:
import os                                     
import google.colab                                                             
import sys

# mounting Google Drive in the runtime's virtual machine

if not os.path.isdir('/content/drive'):

    google.colab.drive.mount('/content/drive') 

In [17]:
os.chdir('/content/drive/MyDrive/Research 2021/G9 1920392_Zannat Chowdhury_1921371_Md Zakaria Kabir/Development')

In [18]:
!ls

Audio_Segmentation.ipynb  playlist_collaboration_link.gdoc


#**Installation & Imports**

**Standard Imports**

In [19]:
import os
import glob
import json
import re

**Other Imports**

In [20]:
modules = {'librosa': False,
           'pandas': False,
           'audio_metadata': False,
           'yt_dlp': False,
           'numpy': False,
           'soundfile': False,
           'pydub': False,}

In [21]:
while True:

  try:

    import librosa
    modules['librosa'] = True

    import pandas as pd
    modules['pandas'] = True

    import audio_metadata
    modules['audio_metadata'] = True

    import yt_dlp
    modules['yt_dlp'] = True

    import numpy as np
    modules['numpy'] = True

    import soundfile as sf
    modules['soundfile'] = True

    from pydub import AudioSegment 
    from pydub.utils import make_chunks
    modules['pydub'] = True

    print('Successfully Imported!!')

    break

  except Exception:

    if modules['librosa'] == False:

      print('Installing librosa')

      !pip install librosa

      continue

    if modules['pandas'] == False:

      print('Installing pandas')

      !pip install pandas
      
      continue
    
    if modules['audio_metadata'] == False:

      print('Installing audio_metadata')

      !pip install -U audio_metadata
      
      continue

    if modules['yt_dlp'] == False:

      print('Installing yt_dlp')

      !python3 -m pip install -U yt-dlp
      
      continue

    if modules['numpy'] == False:

      print('Installing numpy')

      !pip install numpy
      
      continue

    if modules['soundfile'] == False:

      print('Installing soundfile')

      !pip install soundfile
      
      continue

    if modules['pydub'] == False:

      print('Installing pydub')

      !pip install pydub
      
      continue
    print('Packages Installed Successfully')

Successfully Imported!!


#**Initialization**

In [22]:
root_directory = '/content/drive/MyDrive/Research 2021/G9 1920392_Zannat Chowdhury_1921371_Md Zakaria Kabir/Development'

input_directory = os.path.join(root_directory, 'input_directory')

os.makedirs(input_directory, exist_ok=True)

source_directory = os.path.join(input_directory, 'source_directory')

os.makedirs(source_directory, exist_ok=True)

trimmed_output = os.path.join(source_directory, 'trimmed_output')

os.makedirs(trimmed_output, exist_ok=True)

segmented_output = os.path.join(source_directory, 'segmented_output')

os.makedirs(segmented_output, exist_ok=True)
# dialect_list = ['bogura', 'chittagong', 'dinajpur', 'dhaka_old', 'comilla', 'chapai_nawabganj',
#  'barishal', 'bagherhat', 'rajshahi', 'jessore', 'khulna', 'kolkata' 'kushtia',
#  'manikganj', 'mymensingh', 'noakhali', 'rangpur', 'shatkhira', 'sirajgonj',
#  'sylhet']

# **Audio download**

In [23]:
audio_url = {'#bogura':'https://www.youtube.com/playlist?list=PLh79TXh5wUA2sESaJxMUA-yKB753mnLpV',
            '#chittagong':'https://youtube.com/playlist?list=PLh79TXh5wUA1JJdT_o9fNuTuwRcW11wuJ',
            '#dinajpur':'https://www.youtube.com/playlist?list=PLh79TXh5wUA2iTmaHihqGJJXIPVrEvhtB',
            '#dhaka_old':'https://www.youtube.com/playlist?list=PLh79TXh5wUA2oivE_qqlZnhAdbZhJJ8nc',
            '#comilla':'https://www.youtube.com/playlist?list=PLh79TXh5wUA0W1NGAHkRxyHUSB9pb7K8O',
            '#chapai_nawabganj':'https://www.youtube.com/playlist?list=PLh79TXh5wUA3fNAJ9mGUVBF9gaPsYuCwb',
            '#barishal':'https://www.youtube.com/playlist?list=PLh79TXh5wUA1a9gvYcvnGDU2zT56OfzGj',
            '#bagherhat':'https://www.youtube.com/playlist?list=PLh79TXh5wUA0Uw-OvaVis7smYIxua_Q10',
            '#rajshahi':'https://www.youtube.com/playlist?list=PLh79TXh5wUA2dTraJg928KGAKNSaaY-2G',
            '#jessore':'https://www.youtube.com/playlist?list=PLh79TXh5wUA0RGss5T1EvrpJRxHXrr0_S',
            '#khulna':'https://www.youtube.com/playlist?list=PLh79TXh5wUA3tgiCsMq0ADkk4qYtLYMZT',
            '#kolkata':'https://www.youtube.com/playlist?list=PLh79TXh5wUA1X_brXuh6dCrB0Dd3USmwm',
            '#kushtia':'https://www.youtube.com/playlist?list=PLh79TXh5wUA2KFsndexCHzpxvOapGcyKe',
            '#manikganj':'https://www.youtube.com/playlist?list=PLh79TXh5wUA2irbH3RgbTO0dx4oxcsKPF',
            '#mymensingh':'https://www.youtube.com/playlist?list=PLh79TXh5wUA0-zv6ZZHMTyGwx0_iRrnWD',
            '#noakhali':'https://www.youtube.com/playlist?list=PLh79TXh5wUA1xJCfr-3YmZKKKpGQqopKG',
            '#rangpur':'https://www.youtube.com/playlist?list=PLh79TXh5wUA0xBXftNohSOmK6aDJG2DOn',
            '#shatkhira':'https://www.youtube.com/playlist?list=PLh79TXh5wUA3LjmFCWymquAm0ER037AYu',
            '#sirajgonj':'https://www.youtube.com/playlist?list=PLh79TXh5wUA0mfmyXNRkQRggLneN73cM0',
            '#sylhet':'https://www.youtube.com/playlist?list=PLh79TXh5wUA22CAiJs8p_p7MssS4kueRe'}

In [24]:
filepath = os.path.join(source_directory, 'playlist.txt')

with open(filepath, 'w+') as file:

  for key, value in audio_url.items():

    file.write('{0}\n{1}\n'.format(key, value))

In [25]:
!yt-dlp -f 'ba' -x --audio-format wav --download-archive '{source_directory}/downloaded_list.txt' -o '{source_directory}/%(playlist)s/%(playlist)s%(playlist_index)05d.%(ext)s' -a '{filepath}'

[youtube:tab] PLh79TXh5wUA2sESaJxMUA-yKB753mnLpV: Downloading webpage
[youtube:tab] PLh79TXh5wUA2sESaJxMUA-yKB753mnLpV: Downloading API JSON with unavailable videos
[download] Downloading playlist: bogura
[youtube:tab] Playlist bogura: Downloading 8 videos of 8
[download] Downloading video 1 of 8
[youtube] Myn4dipvjec: Downloading webpage
[youtube] Myn4dipvjec: Downloading android player API JSON
[youtube] Myn4dipvjec: Downloading MPD manifest
[youtube] Myn4dipvjec: Downloading MPD manifest
[info] Myn4dipvjec: Downloading 1 format(s): 251
[download] Destination: /content/drive/MyDrive/Research 2021/G9 1920392_Zannat Chowdhury_1921371_Md Zakaria Kabir/Development/input_directory/source_directory/bogura/bogura00001.webm
[download] 100% of 3.14MiB in 00:00 at 3.38MiB/s
[ExtractAudio] Destination: /content/drive/MyDrive/Research 2021/G9 1920392_Zannat Chowdhury_1921371_Md Zakaria Kabir/Development/input_directory/source_directory/bogura/bogura00001.wav
Deleting original file /content/drive

#**Audio Description**

In [26]:
def get_audio_info(dialect_list, dir):

  audio_info = []

  for dialect in dialect_list:

    print('Current Dialect: {0}'.format(dialect))

    current_dialect_path = os.path.join(dir, dialect)

    for audio_path in glob.glob(os.path.join(current_dialect_path, '*.wav')):

      data, sampling_rate = librosa.load(audio_path, sr = None, mono=True)

      metadata = audio_metadata.load(audio_path)

      audio_info.append({'dialect': dialect,
                         'filename': audio_path.split('/')[-1],
                         'sampling_rate': sampling_rate,
                         'duration' : librosa.get_duration(y = data, sr = sampling_rate),
                         'audio_shape' : data.shape,
                         'n_channels': len(data.shape),
                         'filesize': f'{metadata.filesize/1024:.2f} KiB',
                         'bit_rate': f'{metadata.streaminfo.bitrate/1000:.1f} Kbps',
                         'bit_depth': metadata.streaminfo.bit_depth,
                         'filepath' : audio_path
                    })
      
  return pd.DataFrame(audio_info)

In [27]:
dialect_list = [dialect.replace('#', '') for dialect in audio_url.keys()]

audio_info_df = get_audio_info(dialect_list, source_directory)

Current Dialect: bogura
Current Dialect: chittagong
Current Dialect: dinajpur
Current Dialect: dhaka_old
Current Dialect: comilla
Current Dialect: chapai_nawabganj
Current Dialect: barishal
Current Dialect: bagherhat
Current Dialect: rajshahi
Current Dialect: jessore
Current Dialect: khulna
Current Dialect: kolkata
Current Dialect: kushtia
Current Dialect: manikganj
Current Dialect: mymensingh
Current Dialect: noakhali
Current Dialect: rangpur
Current Dialect: shatkhira
Current Dialect: sirajgonj
Current Dialect: sylhet


In [28]:
audio_info_df

,dialect,filename,sampling_rate,duration,audio_shape,n_channels,filesize,bit_rate,bit_depth,filepath
0,bogura,bogura00001.wav,48000,197.445792,"(9477398,)",1,37021.16 KiB,1536.0 Kbps,16,/content/drive/MyDrive/Research 2021/G9 192039...
1,bogura,bogura00002.wav,48000,261.857937,"(12569181,)",1,49098.44 KiB,1536.0 Kbps,16,/content/drive/MyDrive/Research 2021/G9 192039...
2,bogura,bogura00003.wav,48000,167.921604,"(8060237,)",1,31485.38 KiB,1536.0 Kbps,16,/content/drive/MyDrive/Research 2021/G9 192039...
3,bogura,bogura00004.wav,48000,186.822646,"(8967487,)",1,35029.32 KiB,1536.0 Kbps,16,/content/drive/MyDrive/Research 2021/G9 192039...
4,bogura,bogura00005.wav,48000,322.229812,"(15467031,)",1,60418.17 KiB,1536.0 Kbps,16,/content/drive/MyDrive/Research 2021/G9 192039...
...,...,...,...,...,...,...,...,...,...,...
100,sylhet,sylhet00009.wav,48000,77.723708,"(3730738,)",1,14573.27 KiB,1536.0 Kbps,16,/content/drive/MyDrive/Research 2021/G9 192039...
101,sylhet,sylhet00010.wav,48000,1426.779833,"(68485432,)",1,267521.29 KiB,1536.0 Kbps,16,/content/drive/MyDrive/Research 2021/G9 192039...
102,sylhet,sylhet00011.wav,48000,765.417479,"(36740039,)",1,143515.85 KiB,1536.0 Kbps,16,/content/drive/MyDrive/Research 2021/G9 192039...
103,sylhet,sylhet00012.wav,48000,192.093583,"(9220492,)",1,36017.62 KiB,1536.0 Kbps,16,/content/drive/MyDrive/Research 2021/G9 192039...


In [29]:
# final product of this section will be a pandas dataframe

# features, 

# how many channels, 

#**Audio Triming**

**Initializing Trimming Points** 

In [30]:
trimming_points = {}

for path in audio_info_df['filename']:
  
  trimming_points[path] = None

**Write into file to note down the points**

In [31]:
trim_points_filename = os.path.join(source_directory, 'trim_points.txt')

if not os.path.isfile(trim_points_filename) :

  fp = open(trim_points_filename, 'x')

  fp.close()

if os.stat(trim_points_filename).st_size == 0:

  json.dump(trimming_points, open(trim_points_filename,'w'),indent=2, sort_keys=True)

else:

  trim_points_dict = json.load(open(os.path.join(source_directory, 'trim_points.txt')))

  if not trim_points_dict.keys()==trimming_points.keys():

    trimming_points.update(trim_points_dict)

    json.dump(trimming_points, open(trim_points_filename,'w'), indent=2, sort_keys=True)

**Read file to read the trimming points & trim, & save to trimed directory by same file and folder notation**

In [32]:
def trim_audio_dataset(trim_points_filename, trimmed_output):

  trim_points_dict = json.load(open(trim_points_filename))

  for key, points in trim_points_dict.items():

    current_dialect = re.split('\d+', key)[0]

    current_filepath = os.path.join(source_directory, current_dialect, key)

    current_file_output_directory = os.path.join(trimmed_output, current_dialect)

    os.makedirs(current_file_output_directory, exist_ok=True)
    
    audio, sr = librosa.load(current_filepath, sr = None, mono = True)

    buffer = 0

    if points is not None:

      for current_point in points:

        current_point = np.array(current_point) - buffer

        audio = np.delete(audio,slice(current_point[0]*sr,-1 if current_point[1]<0 else current_point[1]*sr))

        buffer += np.diff(current_point)

    sf.write(os.path.join(current_file_output_directory, key), audio, sr)

    print(os.path.join(current_file_output_directory, key))

In [33]:
trim_audio_dataset(trim_points_filename, trimmed_output)

/content/drive/MyDrive/Research 2021/G9 1920392_Zannat Chowdhury_1921371_Md Zakaria Kabir/Development/input_directory/source_directory/trimmed_output/barishal/barishal00002.wav
/content/drive/MyDrive/Research 2021/G9 1920392_Zannat Chowdhury_1921371_Md Zakaria Kabir/Development/input_directory/source_directory/trimmed_output/barishal/barishal00003.wav
/content/drive/MyDrive/Research 2021/G9 1920392_Zannat Chowdhury_1921371_Md Zakaria Kabir/Development/input_directory/source_directory/trimmed_output/barishal/barishal00004.wav
/content/drive/MyDrive/Research 2021/G9 1920392_Zannat Chowdhury_1921371_Md Zakaria Kabir/Development/input_directory/source_directory/trimmed_output/barishal/barishal00005.wav
/content/drive/MyDrive/Research 2021/G9 1920392_Zannat Chowdhury_1921371_Md Zakaria Kabir/Development/input_directory/source_directory/trimmed_output/barishal/barishal00006.wav
/content/drive/MyDrive/Research 2021/G9 1920392_Zannat Chowdhury_1921371_Md Zakaria Kabir/Development/input_directo

In [34]:
# trim_points_dict = json.load(open(os.path.join(source_directory, 'trim_points.txt')))
# trimed_files_directory=os.path.join(source_directory, 'trimed_files')
# os.makedirs(trimed_files_directory, exist_ok=True)

# for key, points in trim_points_dict.items():
#   dialect=re.split('\d+', key)[0]
#   old_file_name=os.path.join(source_directory, dialect,key)
#   trimmed_files_subDir=os.path.join(trimed_files_directory, dialect)
#   os.makedirs(trimmed_files_subDir, exist_ok=True)
  
#   audio, sr = librosa.load(old_file_name, sr = None, mono=True)
#   buffer=0
#   if points is not None:
#     for point in points:
#       point=np.array(point)-buffer
#       audio = np.delete(audio,slice(point[0]*sr,-1 if point[1]<0 else point[1]*sr))
#       buffer += np.diff(point)
#   sf.write(trimmed_files_subDir+'/'+key, audio, sr)
#   print(trimmed_files_subDir+'/'+key)

  

In [35]:
# find the module that can cut a portio of an audio using starting and ending timestamp

# Audio Splitting

In [36]:
def split_audio_into_segments(dialect_list, dir, segment_time=15):

  segment_time = int(input("Enter segment time (in Seconds):"))

  for dialect in dialect_list:

    print('Current Dialect: {0}'.format(dialect))

    current_dialect_path = os.path.join(dir, dialect)
    
    current_file_output_directory = os.path.join(segmented_output, dialect)

    os.makedirs(current_file_output_directory, exist_ok=True)

    count=0

    for audio_path in glob.glob(os.path.join(current_dialect_path, '*.wav')):

      myaudio = AudioSegment.from_file(audio_path, "wav") 

      chunk_length_ms = segment_time*1000

      chunks = make_chunks(myaudio,chunk_length_ms)

      for chunk in chunks: 

        count+=1

        chunk_name = os.path.join(current_file_output_directory, dialect+"{:010}.wav".format(count))

        # print ("exporting", chunk_name) 
        
        chunk.export(chunk_name, format="wav") 


In [37]:
split_audio_into_segments(dialect_list, trimmed_output)

Enter segment time (in Seconds):15
Current Dialect: bogura
Current Dialect: chittagong
Current Dialect: dinajpur
Current Dialect: dhaka_old
Current Dialect: comilla
Current Dialect: chapai_nawabganj
Current Dialect: barishal
Current Dialect: bagherhat
Current Dialect: rajshahi
Current Dialect: jessore
Current Dialect: khulna
Current Dialect: kolkata
Current Dialect: kushtia
Current Dialect: manikganj
Current Dialect: mymensingh
Current Dialect: noakhali
Current Dialect: rangpur
Current Dialect: shatkhira
Current Dialect: sirajgonj
Current Dialect: sylhet


In [38]:
# x=np.array([1,2,3,4,5,6,7,8,9,10])
# num_sections = int(np.ceil(len(x) / 3))
# print(num_sections)
# if len(x)/num_sections>1:
#   print(np.split(x[:int(np.floor(len(x) / 3))*3], 3))
#   print(x[int(np.floor(len(x) / 3))*3:])
#   # print()

In [39]:
# from pydub import AudioSegment 
# from pydub.utils import make_chunks
# import os

# # def process_sudio(file_name):
# #     myaudio = AudioSegment.from_file(file_name, "wav") 
# #     chunk_length_ms = 8000 # pydub calculates in millisec 
# #     chunks = make_chunks(myaudio,chunk_length_ms) #Make chunks of one sec 
# #     for i, chunk in enumerate(chunks): 
# #         chunk_name = './chunked/' + file_name + "_{0}.wav".format(i) 
# #         print ("exporting", chunk_name) 
# #         chunk.export(chunk_name, format="wav") 

# # all_file_names = os.listdir()
# # try:
# #     os.makedirs('chunked') # creating a folder named chunked
# # except:
# #     pass
# # for each_file in all_file_names:
# #     if ('.wav' in each_file):
# #         process_sudio(each_file)
# file_name='/content/drive/MyDrive/Research 2021/G9 1920392_Zannat Chowdhury_1921371_Md Zakaria Kabir/Development/input_directory/source_directory/trimmed_output/bogura/bogura00001.wav'
# myaudio = AudioSegment.from_file(file_name, "wav") 
# chunk_length_ms = 15000 # pydub calculates in millisec 
# chunks = make_chunks(myaudio,chunk_length_ms) #Make chunks of one sec 
# for i, chunk in enumerate(chunks): 
#     chunk_name = file_name + "_{0}.wav".format(i) 
#     print ("exporting", chunk_name) 
#     chunk.export(chunk_name, format="wav") 